In [1]:
import pymongo
import pandas as pd
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
import re
companies = db['companies']

In [2]:
def convert_companies(company):
    return {
        'name': company['name'],
        'category_code': company['category_code'],
        'founded_year': company['founded_year'],
        'number_of_employees': company['number_of_employees'],
        'total_money_raised': company['total_money_raised'],
        'offices': company['offices']
               }

cursor = companies.find()
geoCompanies = list(map(lambda r: convert_companies(r), cursor))

In [3]:
def convert_offices(company):
    companies = []
    for x in geoCompanies:
        for y in x['offices']:
            z = {
            'name': x['name'],
            'category_code': x['category_code'],
            'founded_year': x['founded_year'],
            'number_of_employees': x['number_of_employees'],
            'total_money_raised': x['total_money_raised'],
            'city': y['city'],
            'country_code': y['country_code'],
            'latitude': y['latitude'],
            'longitude': y['longitude']}
            companies.append(z)
    return companies
offices = convert_offices(geoCompanies)

In [4]:
df = pd.DataFrame(offices)
df.head()


,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,total_money_raised
0,enterprise,Pleasanton,USA,1996.0,37.692934,-121.904945,AdventNet,600.0,$0
1,software,Pleasanton,USA,2005.0,37.692934,-121.904945,Zoho,1600.0,$0
2,web,Seattle,USA,2005.0,47.603122,-122.333253,Wetpaint,47.0,$39.8M
3,web,New York,USA,2005.0,40.723731,-73.996431,Wetpaint,47.0,$39.8M
4,news,San Francisco,USA,2004.0,37.764726,-122.394523,Digg,60.0,$45M


In [5]:
def posicion(x):
    points = []
    for i in range(len(x)):
        lat = x.iloc[i]['latitude']
        lon = x.iloc[i]['longitude']
        point = {'type':'Point', 'coordinates': [lon,lat]}
        points.append(point)
    df['points'] = points
    return df['points']
posicion(df)

0        {'type': 'Point', 'coordinates': [-121.904945,...
1        {'type': 'Point', 'coordinates': [-121.904945,...
2        {'type': 'Point', 'coordinates': [-122.333253,...
3        {'type': 'Point', 'coordinates': [-73.9964312,...
4        {'type': 'Point', 'coordinates': [-122.394523,...
5             {'type': 'Point', 'coordinates': [nan, nan]}
6        {'type': 'Point', 'coordinates': [-122.247573,...
7        {'type': 'Point', 'coordinates': [-118.393064,...
8        {'type': 'Point', 'coordinates': [-118.39417, ...
9        {'type': 'Point', 'coordinates': [-122.169472,...
10       {'type': 'Point', 'coordinates': [-118.379768,...
11       {'type': 'Point', 'coordinates': [-122.154369,...
12       {'type': 'Point', 'coordinates': [-118.446596,...
13       {'type': 'Point', 'coordinates': [-117.081406,...
14       {'type': 'Point', 'coordinates': [-122.404052,...
15       {'type': 'Point', 'coordinates': [-122.419204,...
16            {'type': 'Point', 'coordinates': [nan, nan

In [6]:
def moneylaundry(money):
    res = []
    for x in money:
        x = x.replace('$', '').replace('€', '').replace('£', '').replace('¥', '').replace('C', '').replace('kr', '')
        if 'M' in x:
            x = x.replace('M', '')
            res.append(float(x)*1000000)
        elif 'k' in x:
            x = x.replace('k', '')
            res.append(float(x)*1000)

        elif 'B' in x:
            x = x.replace('B', '')
            res.append(float(x)*1000000000)

        else:
            res.append(float(x))
    return res
df['total_money_raised'] = moneylaundry(df['total_money_raised'])
df.head()

,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,total_money_raised,points
0,enterprise,Pleasanton,USA,1996.0,37.692934,-121.904945,AdventNet,600.0,0.0,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,software,Pleasanton,USA,2005.0,37.692934,-121.904945,Zoho,1600.0,0.0,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,web,Seattle,USA,2005.0,47.603122,-122.333253,Wetpaint,47.0,39800000.0,"{'type': 'Point', 'coordinates': [-122.333253,..."
3,web,New York,USA,2005.0,40.723731,-73.996431,Wetpaint,47.0,39800000.0,"{'type': 'Point', 'coordinates': [-73.9964312,..."
4,news,San Francisco,USA,2004.0,37.764726,-122.394523,Digg,60.0,45000000.0,"{'type': 'Point', 'coordinates': [-122.394523,..."


In [7]:
df.dtypes

category_code           object
city                    object
country_code            object
founded_year           float64
latitude               float64
longitude              float64
name                    object
number_of_employees    float64
total_money_raised     float64
points                  object
dtype: object

In [8]:
df.to_json('comp_with_coords.json', orient='records', lines= True)

In [9]:
def success(x):
    web = (6)
    software = (6)
    advertising = (7)
    mobile = (9)
    games_video = (10)
    enterprise = (2)
    other = (-1)
    ecommerce = (4)
    consulting = (1)
    network_hosting = (6)
    public_relations = (-5)
    search = (1)
    biotech = (4)
    hardware = (7)
    cleantech = (6)
    security = (4)
    analytics = (2)
    semiconductor = (-4)
    social = (3)
    finance = (-2)
    music = (-4)
    education = (-5)
    messaging = (-3)
    news = (-8)
    travel = (1)
    photo_video = (3)
    medical = (6)
    health = (5)
    manufacturing = (6)
    hospitality = (8)
    transportation = (2)
    legal = (5)
    real_estate = (5)
    sports = (6)
    fashion = (-3)
    nanotech = (2)
    automotive = (-4)
    design = (2)
    nonprofit = (-2)
    